# load, predict

In [68]:
import cv2
import os
import time
import numpy as np
import tensorflow as tf
import time
import collections
import PIL
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook as tqdm
import tensorflow.contrib.tensorrt as trt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [2]:
output_names = ['predictions/Softmax']
input_names = ['input_1']

import tensorflow as tf


def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def


trt_graph = get_frozen_graph('./model/trt_graph.pb')

# Create session and load graph
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)

Instructions for updating:
Use tf.gfile.GFile.


In [3]:
tf.import_graph_def(trt_graph, name='')

In [4]:
# Get graph input size
for node in trt_graph.node:
    if 'input_' in node.name:
        size = node.attr['shape'].shape
        image_size = [size.dim[i].size for i in range(1, 4)]
        break
print("image_size: {}".format(image_size))


# input and output tensor names.
input_tensor_name = input_names[0] + ":0"
output_tensor_name = output_names[0] + ":0"

print("input_tensor_name: {}\noutput_tensor_name: {}".format(
    input_tensor_name, output_tensor_name))



image_size: [71, 71, 3]
input_tensor_name: input_1:0
output_tensor_name: predictions/Softmax:0


In [5]:
output_tensor = tf_sess.graph.get_tensor_by_name(output_tensor_name)

In [6]:
output_tensor

<tf.Tensor 'predictions/Softmax:0' shape=<unknown> dtype=float32>

In [193]:
folder = "/home/jetson/datasets/colours/4"
total = np.empty([0, 71, 71, 3])
dim = (71, 71)
for i, file in enumerate(tqdm(os.listdir(folder))):
    if i > 200000:
        break
    #x = cv2.imread(folder + "/" + file)
    #x = cv2.resize(x, dim, interpolation=cv2.INTER_LINEAR)
    x = image.load_img(folder + "/" + file, target_size=dim)
    x = np.array(x)
    x += 50
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    total = np.concatenate((total, x))

In [194]:
total.shape

(2845, 71, 71, 3)

In [195]:
total[0:1][0].shape

(71, 71, 3)

In [196]:
import matplotlib.pyplot as plt
from scipy import misc

ModuleNotFoundError: No module named 'scipy'

In [197]:
img = total[0:1][0]
img.shape

(71, 71, 3)

In [204]:
for i in range(40):
    img = total[i:i+1][0]
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [198]:
start = time.time()
predictions = np.empty((1, 6))
for i in tqdm(range(total.shape[0])):
    t = np.transpose(total[i:i+1], axes=(0, 2, 1, 3))
    feed_dict = {
    input_tensor_name: t# total[i:i+1]
    }
    preds = tf_sess.run(output_tensor, feed_dict)
    predictions = np.concatenate((predictions, preds))
end = time.time()
print(end - start)


13.137482643127441


In [199]:
predictions = predictions[1:]

In [200]:
predictions.shape

(2845, 6)

In [201]:
predictions

array([[1.32674649e-01, 3.59364133e-03, 9.43216310e-06, 1.70179501e-05,
        1.53258955e-03, 8.62172544e-01],
       [9.09857154e-01, 2.73285434e-02, 1.37515717e-05, 2.37697423e-05,
        1.30857574e-03, 6.14682063e-02],
       [4.40866292e-01, 2.61086505e-02, 2.00713566e-05, 3.45101871e-05,
        2.26401817e-03, 5.30706525e-01],
       ...,
       [9.49505866e-01, 1.37673141e-02, 3.26956360e-04, 2.65068869e-04,
        3.53209749e-02, 8.13863764e-04],
       [4.03215352e-04, 3.17101985e-05, 2.13402913e-08, 4.70233168e-08,
        4.32443312e-06, 9.99560654e-01],
       [7.25417770e-03, 7.22746074e-04, 3.72005115e-06, 1.96173278e-05,
        1.43186713e-03, 9.90567863e-01]])

In [202]:
c = collections.Counter(np.argmax(predictions[:, :], axis=1))
c

Counter({5: 839, 0: 1978, 4: 28})